In [56]:
import requests 
import os
from datetime import datetime, timedelta
import json
import pandas as pd
import csv

In [57]:
def get_metadata():
    lines = 10 # 10tk praegu
    with open(f"arxiv-metadata-oai-snapshot.json", 'r') as f:
        for line in f:
            if lines == 0:
                break
            lines -= 1
            yield line

In [58]:
metadata = get_metadata()

authors_parsed = []
categories	 = []
doi = []
ids = []
titles = []
versions = []
author_ids = []
authors = []
for paper in metadata:
    metaDict = json.loads(paper)

    authorsp = metaDict['authors_parsed']
    a_ids = []
    for a in authorsp:
        author_id = "_".join(a)  
        a_ids.append(author_id)    
        authors.append({
                "authorId": author_id, 
                "first_name": a[1],
                "last_name": a[0]
            })

    ids.append(metaDict['id'])
    doi.append(metaDict['doi'])
    titles.append(metaDict['title'])
    authors_parsed.append(metaDict['authors_parsed'])
    author_ids.append(a_ids)
    categories.append(metaDict['categories'])
    versions.append(metaDict['versions'])


df = pd.DataFrame({'id' : ids, 'doi': doi, 'title' : titles, 'authors_parsed': authors_parsed, 'author_ids': author_ids, 'categories' : categories, 'versions': versions})
author_df = pd.DataFrame.from_records(authors)

# Remove the duplicates
df = df[~df.title.duplicated()]

# Enforcing types
df['doi']= df['doi'].astype('str')
df['id']= df['id'].astype('str')
df['title']= df['title'].astype('str')
df['authors_parsed']= df['authors_parsed'].astype('str')
df['categories']= df['categories'].astype('str')
df['versions']= df['versions'].astype('str')

In [59]:
author_df = author_df[~author_df.first_name.str.contains(' ')]

In [60]:
author_df = author_df[~author_df.first_name.str.contains('C.')]

In [61]:
author_df

,authorId,first_name,last_name
4,Streinu_Ileana_,Ileana,Streinu
5,Theran_Louis_,Louis,Theran
6,Pan_Hongjun_,Hongjun,Pan
7,Callan_David_,David,Callan
8,Abu-Shammala_Wael_,Wael,Abu-Shammala
9,Torchinsky_Alberto_,Alberto,Torchinsky
12,Corichi_Alejandro_,Alejandro,Corichi
13,Vukasinac_Tatjana_,Tatjana,Vukasinac
16,Harvey_Paul_,Paul,Harvey
17,Merin_Bruno_,Bruno,Merin


In [62]:
def find_genders(first_name, last_name):
    try:
        url = f'https://innovaapi.aminer.cn/tools/v1/predict/gender?name={first_name}+{last_name}&org='
        r = requests.get(url=url)
        data = r.json()
        gender = data['data']['Final']['gender']
    except:
        gender = "UNKNOWN"
    return gender

In [63]:
genders = [find_genders(f, l) for f, l in zip(author_df['first_name'], author_df['last_name'])]
author_df['gender'] = genders
author_df

,authorId,first_name,last_name,gender
4,Streinu_Ileana_,Ileana,Streinu,female
5,Theran_Louis_,Louis,Theran,male
6,Pan_Hongjun_,Hongjun,Pan,UNKNOWN
7,Callan_David_,David,Callan,male
8,Abu-Shammala_Wael_,Wael,Abu-Shammala,male
9,Torchinsky_Alberto_,Alberto,Torchinsky,male
12,Corichi_Alejandro_,Alejandro,Corichi,male
13,Vukasinac_Tatjana_,Tatjana,Vukasinac,female
16,Harvey_Paul_,Paul,Harvey,male
17,Merin_Bruno_,Bruno,Merin,male
